In [2]:
import os
import time
import pandas as pd
import shutil
import pathlib
import random
import datetime as dt
import csv
import numpy as np
import gzip
from matplotlib import pyplot as plt
import copy
import seaborn as sns
from tqdm.notebook import tqdm
import glob
import nibabel as nib
import torch
from nibabel.processing import conform
from tqdm.notebook import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.model_selection import cross_validate
from xgboost import XGBRegressor
import scipy

random.seed(1995)

DATA_BASE='/home/guests/paul_hager/Projects/petgan/data'
ADNI_DATA_BASE='/home/guests/projects/adni/data_bids_processed'
PROJECT_BASE='/home/guests/paul_hager/Projects/ATN_Classification'
SAVE = False

# Functions

In [3]:
def get_diag(row):
    if (row['DXCURREN']==1) or (row['DXCHANGE']==1) or (row['DXCHANGE']==7) or (row['DXCHANGE']==9) or (row['DIAGNOSIS']==1):
        return 'CN'
    elif (row['DXCURREN']==2) or (row['DXCHANGE']==2) or (row['DXCHANGE']==4) or (row['DXCHANGE']==8) or (row['DIAGNOSIS']==2):
        return 'MCI'
    else:
        return 'AD'

In [4]:
def readPaths(pathsFile):
    with open(pathsFile, 'r') as f:
        paths = f.readlines()
    paths = [p.strip() for p in paths]
    return paths

In [5]:
def minmaxNormalize(image, gmin=None, gmax=None):
    image = (image - np.min(image)) / (np.max(image) - np.min(image))
    return image

In [6]:
def zNormalize(image):
    image = (image - np.mean(image[image != np.min(image)])) / np.std(image[image != np.min(image)])
    return image

In [7]:
def zNormalize_torch(image):
    image = (image - torch.mean(image[image != torch.min(image)])) / torch.std(image[image != torch.min(image)])
    return image

In [8]:
def Calc_CL_FBB(FBB_SUVR):
    return (159.08*FBB_SUVR)-151.65

def Calc_CL_AV45(AV45_SUVR):
    return (196.9*AV45_SUVR)-196.03

def Calc_FBB_CL(CL):
    return (CL+151.65)/159.08

def Calc_AV45_CL(CL):
    return (CL+196.03)/196.9

def Calc_FBB_From_AV45(AV45_SUVR):
    return (1.2377*AV45_SUVR) - 0.279

def Calc_AV45_From_FBB(FBB_SUVR):
    return (0.808*FBB_SUVR) + 0.225393

In [9]:
def calc_hipp_vol(row):
    regions = [4001, 4002, 4011, 4012, 4021, 4022, 4101, 4102, 4111, 4112, 4201, 4202] # Entire medial temporal lobe
    regions = [4101,4102] # Hippocampus only
    total = 0
    for l in regions:
        total += row[l]
    return total

In [10]:
abetaDKTLabels = [1003, 1012, 1014, 1018, 1019, 1020, 
    1027, 1028, 2003, 2012, 2014, 2018, 
    2019, 2020, 2027, 2028, 1002, 1010, 
    1023, 1026, 2002, 2010, 2023, 2026, 
    1008, 1025, 1029, 1031, 2008, 2025, 
    2029, 2031, 1015, 1030, 2015, 2030]
def Calc_Amyloid_Sensitive_SUVR_dkt(row):
    avg_suvr = 0
    for i in abetaDKTLabels:
        avg_suvr += row['SUVR.DKT.ROI.idx.{}_abeta'.format(i)]
    avg_suvr = avg_suvr / len(abetaDKTLabels)
    return avg_suvr

In [11]:
# Weighted splitter that controls for bleeding
class ContinueI(Exception):
    pass

def split_groups(out_df, num_splits=5):
    random.seed(1995)
    continue_i = ContinueI()

    n = len(out_df)
    all_groups = []
    all_seen_subs = []
    num_splits = num_splits
    for i in range(num_splits):
        all_groups.append([])
        all_seen_subs.append([])
    for indx, r in out_df.iterrows():
        sub = r['RID']
        try:
            for i,seen_subs in enumerate(all_seen_subs):
                if sub in seen_subs:
                    all_groups[i].append(indx)
                    raise continue_i
        except ContinueI:
            continue
        if sum([len(all_groups[k]) for k in range(num_splits)]) > 0.8*n: # Last 20% should be filled acording to smallest groups for even split
            smallest_group = float('inf')
            i = num_splits
            for j in range(num_splits):
                if len(all_groups[j])<smallest_group:
                    smallest_group = len(all_groups[j])
                    i = j
        else:
            wghts = [50000*(1-len(g)/n) for g in all_groups]
            i = random.choices(list(range(num_splits)),weights=wghts,k=1)[0]
        all_groups[i].append(indx)
        all_seen_subs[i].append(sub)

    for g in all_groups:
        print(len(g))
        
    return all_groups

In [12]:
commonLabels = [2001, 2002, 2101, 2102, 2111, 2112, 2201, 2202, 2211, 2212, 2301, 2302, 2311, 2312, 2321, 2322, 2331, 2332, 2401, 2402, 2501, 2502, 2601, 2602, 2611, 2612, 2701, 2702, 3001, 3002, 4001, 4002, 4011, 4012, 4021, 4022, 4101, 4102, 4111, 4112, 4201, 4202, 5001, 5002, 5011, 5012, 5021, 5022, 5101, 5102, 5201, 5202, 5301, 5302, 5401, 5402, 6001, 6002, 6101, 6102, 6201, 6202, 6211, 6212, 6221, 6222, 6301, 6302, 6401, 6402, 7001, 7002, 7011, 7012, 7021, 7022, 7101, 7102, 8101, 8102, 8111, 8112, 8121, 8122, 8201, 8202, 8211, 8212, 8301, 8302, 9001, 9002, 9022, 9031, 9032, 9041, 9042, 9120]
dktLabels = [2,4,5,7,8,10,11,12,13,14,15,16,17,18,24,26,28,30,31,41,43,44,46,47,49,50,51,52,53,54,58,60,62,63,77,85,251,252,253,254,255,1000,1002,1003,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1034,1035,2000,2002,2003,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2034,2035]

In [13]:
def evaluate_performance(model, X, y, X_test, y_test, threshold_val, _print=True):
    start = time.time()
    y_hat = model.predict(X)
    print(f'Prediction time was {time.time()-start}')
    f1 = metrics.f1_score(y, y_hat)
    auc = metrics.roc_auc_score(y, y_hat)
    if _print:
        print("Train")
        print('F1: {}'.format(f1))
        print('AUC: {}'.format(auc))
        print("---")

    y_hat_test = model.decision_function(X_test)
    auc_test = metrics.roc_auc_score(y_test, y_hat_test)

    pr, re, th = metrics.precision_recall_curve(y_test,y_hat_test)
    f1 = scipy.stats.hmean([pr,re])
    
    _min=1000
    for i, t in enumerate(th):
        if abs(t-threshold_val)<_min:
            indx = i
            _min = abs(t-threshold_val)
    precision_test = pr[indx]
    recall_test = re[indx]
    f1_test = f1[indx]


    if _print:
        print("Test")
        print('F1: {} | Precision: {} | Recall: {}'.format(f1_test,precision_test,recall_test))
        print('AUC: {}'.format(auc_test))
    
    return model.decision_function(X_test)>threshold_val, auc_test, f1_test, precision_test, recall_test

In [14]:
def best_cutoff(y, y_hat):
    pr, re, th = metrics.precision_recall_curve(y,y_hat)
    f1 = scipy.stats.hmean([pr,re])
    best_i = np.argmax(f1)
    threshold = th[best_i]
    return threshold

In [15]:
def classify_atn(n, a):
    out = []
    for i in range(len(n)):
        n_ = n[i]
        a_ = a[i]
        if n_ and a_:
            out.append(3) # N+|A+
        elif n_ and not a_:
            out.append(2) # N+|A-
        elif not n_ and a_:
            out.append(1) # N-|A+
        elif not n_ and not a_:
            out.append(0) # N-|A-
    return out

In [16]:
def calc_CI(var, all_scores):
    scores_numpy = np.array(all_scores[var])
    mean = scores_numpy.mean()
    CI_low = mean - 1.96 * scores_numpy.std() / np.sqrt(len(scores_numpy))
    CI_high = mean + 1.96 * scores_numpy.std() / np.sqrt(len(scores_numpy))
    print(f'{var:20}: {mean:.2f} [{CI_low:.2f}, {CI_high:.2f}]')
    
def calc_mean_std(var, all_scores):
    scores_numpy = np.array(all_scores[var])
    print(f'{var:20}: {scores_numpy.mean():.2f} ± {scores_numpy.std():.2f}')

# Read Files and Tidy

In [17]:
mri_df = pd.read_csv(os.path.join(DATA_BASE,'ADNI_Search','MRI_ADNI_Search.csv'))
mri_df['Study Date'] = pd.to_datetime(mri_df['Study Date'])
allSequences = pd.unique(mri_df['Description'])

T1_Sequences = [v for i,v in enumerate(allSequences) if ('mprage' in v.lower() or 'mp-rage' in v.lower() or 'spgr' in v.lower()) and 'repe' not in v.lower()]
t1_df_dup = mri_df[mri_df['Description'].isin(T1_Sequences)]
t1_df = t1_df_dup.drop_duplicates(subset=['Subject ID','Study Date'])
t1_scanners_df = t1_df[t1_df['Imaging Protocol'].str.contains('Mfg Model').notna()]
t1_scanners_df['Scanner Manufacturer'] = t1_scanners_df.apply(lambda row: row['Imaging Protocol'].split(';')[0].split('=')[1], axis=1)
t1_scanners_df['Scanner Model'] = t1_scanners_df.apply(lambda row: row['Imaging Protocol'].split(';')[1].split('=')[1], axis=1)

print("There are {} unique T1 scans from {} subjects".format(len(t1_df),len(pd.unique(t1_df['Subject ID']))))

FLAIR_Sequences = [v for i,v in enumerate(allSequences) if 'flair' in v.lower()]
FLAIR_Sequences_3D = [v for v in FLAIR_Sequences if '3d' in v.lower()]
flair_df_dup = mri_df[mri_df['Description'].isin(FLAIR_Sequences)]
flair_df = flair_df_dup.drop_duplicates(subset=['Subject ID','Study Date'])
mri_3D_flair_df_dup = mri_df[mri_df['Description'].isin(FLAIR_Sequences_3D)]
mri_3D_flair_df = mri_3D_flair_df_dup.drop_duplicates(subset=['Subject ID','Study Date'])
print("There are {} unique FLAIRS from {} subjects".format(len(flair_df),len(pd.unique(flair_df['Subject ID']))))
print("There are {} unique 3D FLAIRS from {} subjects".format(len(mri_3D_flair_df),len(pd.unique(mri_3D_flair_df['Subject ID']))))

There are 11100 unique T1 scans from 2533 subjects
There are 6365 unique FLAIRS from 1809 subjects
There are 1905 unique 3D FLAIRS from 1052 subjects


/tmp/ipykernel_4115128/2497137033.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1_scanners_df['Scanner Manufacturer'] = t1_scanners_df.apply(lambda row: row['Imaging Protocol'].split(';')[0].split('=')[1], axis=1)
/tmp/ipykernel_4115128/2497137033.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1_scanners_df['Scanner Model'] = t1_scanners_df.apply(lambda row: row['Imaging Protocol'].split(';')[1].split('=')[1], axis=1)


In [18]:
pet_df = pd.read_csv(os.path.join(DATA_BASE,'ADNI_Search','PET_ADNI_Search.csv'))
pet_df['Study Date'] = pd.to_datetime(pet_df['Study Date'])
allSequences_PET = pd.unique(pet_df['Imaging Protocol'])

tau_sequences = [v for i,v in enumerate(allSequences_PET) if 'av1451' in v.lower()]
tau_df_dup = pet_df[pet_df['Imaging Protocol'].isin(tau_sequences)]
tau_df = tau_df_dup.drop_duplicates(subset=['Subject ID','Study Date'])
print("There are {} unique tau scans from {} subjects".format(len(tau_df),len(pd.unique(tau_df['Subject ID']))))

abeta_sequences = [v for i,v in enumerate(allSequences_PET) if 'av45' in v.lower() or 'fbb' in v.lower()]
abeta_df_dup = pet_df[pet_df['Imaging Protocol'].isin(abeta_sequences)]
abeta_df = abeta_df_dup.drop_duplicates(subset=['Subject ID','Study Date'])
print("There are {} unique abeta scans from {} subjects".format(len(abeta_df),len(pd.unique(abeta_df['Subject ID']))))

fdg_sequences = [v for i,v in enumerate(allSequences_PET) if 'fdg' in v.lower()]
fdg_df_dup = pet_df[pet_df['Imaging Protocol'].isin(fdg_sequences)]
fdg_df = fdg_df_dup.drop_duplicates(subset=['Subject ID','Study Date'])
print("There are {} unique fdg scans from {} subjects".format(len(fdg_df),len(pd.unique(fdg_df['Subject ID']))))


There are 1373 unique tau scans from 870 subjects
There are 3529 unique abeta scans from 1677 subjects
There are 3737 unique fdg scans from 1662 subjects


In [19]:
diag_df = pd.read_csv(os.path.join(DATA_BASE,'ADNI_Search','DXSUM_PDXCONV_ADNIALL.csv'))
diag_df['EXAMDATE'] = pd.to_datetime(diag_df['EXAMDATE'])
diag_df = diag_df.rename(columns={'EXAMDATE':'Study Date', 'PTID':'Subject ID'})
diag_df = diag_df.dropna(subset=['Study Date'])
print("There are {} unique diagnoses from {} subjects".format(len(diag_df),len(pd.unique(diag_df['RID']))))

There are 12842 unique diagnoses from 2853 subjects


In [20]:
biomark1_df = pd.read_csv(os.path.join(DATA_BASE,'ADNI_Search','UPENNBIOMK9_04_19_17.csv'), usecols=['RID', 'EXAMDATE', 'ABETA', 'PTAU', 'TAU'])
biomark1_df = biomark1_df.rename(columns={'ABETA':'ABETA42'})
biomark2_df = pd.read_csv(os.path.join(DATA_BASE,'ADNI_Search','UPENNBIOMK10_07_29_19.csv'), usecols=['RID', 'DRAWDATE', 'ABETA40', 'ABETA42', 'PTAU', 'TAU'])
biomark2_df = biomark2_df.rename(columns={'DRAWDATE':'EXAMDATE'})
biomark3_df = pd.read_csv(os.path.join(DATA_BASE,'ADNI_Search','UPENNBIOMK12_01_04_21.csv'), usecols=['RID', 'EXAMDATE', 'AB40', 'ABETA', 'PTAU', 'TAU'])
biomark3_df = biomark3_df.rename(columns={'AB40':'ABETA40','ABETA':'ABETA42'})

biomark_df = pd.concat([biomark1_df,biomark2_df,biomark3_df])
biomark_df = biomark_df.reset_index(drop=True)
biomark_df = biomark_df.rename(columns={'EXAMDATE':'CSF Date'})
biomark_df['CSF Date'] = pd.to_datetime(biomark_df['CSF Date'])
biomark_df['RID'] = pd.to_numeric(biomark_df['RID'])
biomark_df = biomark_df.replace('>120','121')
biomark_df = biomark_df.replace('<8','7')
biomark_df = biomark_df.replace('>1700','1701')
biomark_df = biomark_df.replace('>1300','1301')
biomark_df = biomark_df.replace('<200','199')
biomark_df = biomark_df.replace('<80','80')
biomark_df['PTAU'] = pd.to_numeric(biomark_df['PTAU'])
biomark_df['TAU'] = pd.to_numeric(biomark_df['TAU'])
biomark_df['ABETA40'] = pd.to_numeric(biomark_df['ABETA40'])
biomark_df['ABETA42'] = pd.to_numeric(biomark_df['ABETA42'])
print("There are {} unique biomark readings from {} subjects".format(len(biomark_df),len(pd.unique(biomark_df['RID']))))

There are 3115 unique biomark readings from 1613 subjects


# Get Overlap

In [21]:
# Find overlap between dataframes
def get_overlap_dataframe(df1,df2,df1_name='1',df2_name='2',df1_date_name='Study Date',df2_date_name='Study Date',df1_id_name='Subject ID',df2_id_name='Subject ID',days_delta=183):
    df1_to_df2_index = {}
    for indx, df1_r in df1.iterrows():
        df2_filt = df2[df2[df2_id_name]==df1_r[df1_id_name]]
        bestDif = dt.timedelta(days=99999)
        for indx2, df2_r in df2_filt.iterrows():
            dif = abs(df1_r[df1_date_name]-df2_r[df2_date_name])
            if dif<dt.timedelta(days=days_delta) and dif<bestDif:
                bestDif = dif
                df1_to_df2_index[indx] = indx2
    df1_filtered_df = df1.loc[list(df1_to_df2_index.keys())]
    df2_filtered_df = df2.loc[list(df1_to_df2_index.values())]
    for c in ['Study Date','Description','Imaging Protocol','Image ID']:
        df1_filtered_df = df1_filtered_df.rename(columns={c:'{}_{}'.format(c,df1_name)})
        df2_filtered_df = df2_filtered_df.rename(columns={c:'{}_{}'.format(c,df2_name)})
    overlap_df = pd.concat([df1_filtered_df.reset_index(drop=True),df2_filtered_df.reset_index(drop=True)], axis=1)
    #for i in [4,6,7,8]:
    #    overlap_df.columns.values[i] = '{}_{}'.format(overlap_df.columns.values[i],df1_name)
    #for i in [13,15,16,17]:
    #    overlap_df.columns.values[i] = '{}_{}'.format(overlap_df.columns.values[i],df2_name)
    overlap_df = overlap_df.loc[:,~overlap_df.columns.duplicated()]
    return overlap_df

In [22]:
# def get_overlap_dataframe(df1,df2,df1_date_name='Study Date',df2_date_name='Study Date',days_delta=183):
t1_tau_overlap_df = get_overlap_dataframe(t1_df,tau_df,df1_name='t1',df2_name='tau')
t1_abeta_overlap_df = get_overlap_dataframe(t1_df,abeta_df,df1_name='t1',df2_name='abeta')
t1_fdg_overlap_df = get_overlap_dataframe(t1_df,fdg_df,df1_name='t1',df2_name='fdg')
tau_abeta_overlap_df = get_overlap_dataframe(tau_df,abeta_df,df1_name='tau',df2_name='abeta')
tau_fdg_overlap_df = get_overlap_dataframe(tau_df,fdg_df,df1_name='tau',df2_name='fdg')
abeta_fdg_overlap_df = get_overlap_dataframe(abeta_df,fdg_df,df1_name='abeta',df2_name='fdg')

In [23]:
print("Overlap between {} and {} is {} scans from {} subjects".format('t1','tau',len(t1_tau_overlap_df),len(pd.unique(t1_tau_overlap_df['Subject ID']))))
print("Overlap between {} and {} is {} scans from {} subjects".format('t1','abeta',len(t1_abeta_overlap_df),len(pd.unique(t1_abeta_overlap_df['Subject ID']))))
print("Overlap between {} and {} is {} scans from {} subjects".format('t1','fdg',len(t1_fdg_overlap_df),len(pd.unique(t1_fdg_overlap_df['Subject ID']))))
print("Overlap between {} and {} is {} scans from {} subjects".format('tau','abeta',len(tau_abeta_overlap_df),len(pd.unique(tau_abeta_overlap_df['Subject ID']))))
print("Overlap between {} and {} is {} scans from {} subjects".format('tau','fdg',len(tau_fdg_overlap_df),len(pd.unique(tau_fdg_overlap_df['Subject ID']))))
print("Overlap between {} and {} is {} scans from {} subjects".format('abeta','fdg',len(abeta_fdg_overlap_df),len(pd.unique(abeta_fdg_overlap_df['Subject ID']))))

Overlap between t1 and tau is 1232 scans from 803 subjects
Overlap between t1 and abeta is 4534 scans from 1635 subjects
Overlap between t1 and fdg is 4827 scans from 1637 subjects
Overlap between tau and abeta is 1072 scans from 823 subjects
Overlap between tau and fdg is 339 scans from 339 subjects
Overlap between abeta and fdg is 1917 scans from 1320 subjects


In [24]:
t1_tau_abeta_overlap_df = get_overlap_dataframe(t1_tau_overlap_df,abeta_df,df1_name='tau',df2_name='abeta',df1_date_name='Study Date_t1')
t1_tau_fdg_overlap_df = get_overlap_dataframe(t1_tau_overlap_df,fdg_df,df1_name='tau',df2_name='fdg',df1_date_name='Study Date_t1')
t1_abeta_fdg_overlap_df = get_overlap_dataframe(t1_abeta_overlap_df,fdg_df,df1_name='abeta',df2_name='fdg',df1_date_name='Study Date_t1')
tau_abeta_fdg_overlap_df = get_overlap_dataframe(tau_abeta_overlap_df,fdg_df,df1_name='abeta',df2_name='fdg',df1_date_name='Study Date_tau')

In [25]:
print("Overlap between {} and {} and {} is {} scans from {} subjects".format('t1','tau','abeta',len(t1_tau_abeta_overlap_df),len(pd.unique(t1_tau_abeta_overlap_df['Subject ID']))))
print("Overlap between {} and {} and {} is {} scans from {} subjects".format('t1','tau','fdg',len(t1_tau_fdg_overlap_df),len(pd.unique(t1_tau_fdg_overlap_df['Subject ID']))))
print("Overlap between {} and {} and {} is {} scans from {} subjects".format('t1','abeta','fdg',len(t1_abeta_fdg_overlap_df),len(pd.unique(t1_abeta_fdg_overlap_df['Subject ID']))))
print("Overlap between {} and {} and {} is {} scans from {} subjects".format('tau','abeta','fdg',len(tau_abeta_fdg_overlap_df),len(pd.unique(tau_abeta_fdg_overlap_df['Subject ID']))))

Overlap between t1 and tau and abeta is 980 scans from 766 subjects
Overlap between t1 and tau and fdg is 312 scans from 311 subjects
Overlap between t1 and abeta and fdg is 3001 scans from 1289 subjects
Overlap between tau and abeta and fdg is 329 scans from 329 subjects


## CSF

In [26]:
import warnings
warnings.simplefilter(action='ignore')
biomark_df = biomark_df.rename(columns={'CSF Date':'Study Date'})
biomark_df['RID'] = biomark_df.apply(lambda row: str(row['RID']).zfill(4),axis=1)
abeta_df['RID'] = abeta_df.apply(lambda row: row['Subject ID'].split('_')[2], axis=1)
t1_df['RID'] = t1_df.apply(lambda row: row['Subject ID'].split('_')[2], axis=1)
abeta_pet_csf_df = pd.concat([abeta_df,biomark_df])
t1_abeta_pet_csf_overlap_df = get_overlap_dataframe(t1_df,abeta_pet_csf_df,df1_name='t1',df2_name='abeta',df1_id_name='RID',df2_id_name='RID')
print("Overlap between {} and {} is {} scans from {} subjects".format('t1','abeta',len(t1_abeta_pet_csf_overlap_df),len(pd.unique(t1_abeta_pet_csf_overlap_df['Subject ID']))))

Overlap between t1 and abeta is 8383 scans from 1960 subjects


## Diagnosis

In [27]:
diag_t1_overlap_df = get_overlap_dataframe(t1_df,diag_df,df2_name='diag',df1_name='t1')
diag_tau_overlap_df = get_overlap_dataframe(tau_df,diag_df,df2_name='diag',df1_name='tau')
diag_abeta_overlap_df = get_overlap_dataframe(abeta_df,diag_df,df2_name='diag',df1_name='abeta')
diag_fdg_overlap_df = get_overlap_dataframe(fdg_df,diag_df,df2_name='diag',df1_name='fdg')

In [28]:
print("Overlap between {} and {} is {} scans from {} subjects".format('diag','t1',len(diag_t1_overlap_df),len(pd.unique(diag_t1_overlap_df['Subject ID']))))
print("Overlap between {} and {} is {} scans from {} subjects".format('diag','tau',len(diag_tau_overlap_df),len(pd.unique(diag_tau_overlap_df['Subject ID']))))
print("Overlap between {} and {} is {} scans from {} subjects".format('diag','abeta',len(diag_abeta_overlap_df),len(pd.unique(diag_abeta_overlap_df['Subject ID']))))
print("Overlap between {} and {} is {} scans from {} subjects".format('diag','fdg',len(diag_fdg_overlap_df),len(pd.unique(diag_fdg_overlap_df['Subject ID']))))

Overlap between diag and t1 is 10864 scans from 2487 subjects
Overlap between diag and tau is 1300 scans from 842 subjects
Overlap between diag and abeta is 3418 scans from 1669 subjects
Overlap between diag and fdg is 3691 scans from 1656 subjects


# Predict Tau and Amyloid from PET

## Create Dataset

In [29]:
suvr_df = pd.read_csv(os.path.join(DATA_BASE,'ADNI_Tau_Amyloid_SUVR_status.csv'))
suvr_df['RID'] = pd.to_numeric(suvr_df['RID'])
suvr_df['acq.date'] = pd.to_datetime(suvr_df['acq.date'])
#tau_abeta_overlap_copy = tau_abeta_overlap_df.loc[~tau_abeta_overlap_df.duplicated(subset=['Subject ID'],keep='first')]
tau_abeta_overlap_copy = tau_abeta_overlap_df.copy(deep=True)
tau_abeta_overlap_copy['RID'] = tau_abeta_overlap_copy.apply(lambda row: int(row['Subject ID'].split('_')[2]), axis=1)
tau_abeta_overlap_suvr_df = tau_abeta_overlap_copy.merge(suvr_df[suvr_df['pet.modality']=='pet-AV1451'], how='left', left_on=['RID','Study Date_tau'], right_on=['RID','acq.date'])
tau_abeta_overlap_suvr_df = tau_abeta_overlap_suvr_df.merge(suvr_df[(suvr_df['pet.modality']=='pet-AV45') | (suvr_df['pet.modality']=='pet-FBB')], how='left', left_on=['RID','Study Date_abeta'], right_on=['RID','acq.date'], suffixes=['_tau','_abeta'])
subset = ['SUVR.DKT.ROI.idx.{}_tau'.format(x) for x in dktLabels]
subset.extend(['SUVR.DKT.ROI.idx.{}_abeta'.format(x) for x in dktLabels])
subset.append('APOE A1')
tau_abeta_overlap_suvr_df = tau_abeta_overlap_suvr_df.dropna(subset=subset)
tau_abeta_overlap_suvr_df['apoe'] = tau_abeta_overlap_suvr_df.apply(lambda row: '{}{}'.format(int(row['APOE A1']),int(row['APOE A2'])), axis=1)
tau_abeta_overlap_suvr_df = tau_abeta_overlap_suvr_df.dropna(subset=subset)
out_df = tau_abeta_overlap_suvr_df

In [30]:
diag_tau_abeta_overlap_df = get_overlap_dataframe(tau_abeta_overlap_suvr_df,diag_df,df1_date_name='Study Date_tau',days_delta=1265)
diag_tau_abeta_overlap_df['Diag'] = diag_tau_abeta_overlap_df.apply(lambda row: get_diag(row),axis=1)

In [31]:
diag_tau_abeta_overlap_df['Diag'].value_counts()

CN     466
MCI    200
AD      70
Name: Diag, dtype: int64

In [32]:
tau_abeta_overlap_suvr_df['T+'] = tau_abeta_overlap_suvr_df.apply(lambda row: 1 if row['tau.global.SUVR_tau']>1.3 else 0, axis=1)
print(tau_abeta_overlap_suvr_df.groupby(["T+"]).size().reset_index().rename(columns={0:'count'}))

tau_abeta_AV45_overlap_suvr_df = tau_abeta_overlap_suvr_df[(tau_abeta_overlap_suvr_df['Description_abeta'].str.contains("AV45"))|(tau_abeta_overlap_suvr_df['Description_abeta'].str.contains("AV_45"))|(tau_abeta_overlap_suvr_df['Description_abeta'].str.contains("AV-45"))]
tau_abeta_FBB_overlap_suvr_df = tau_abeta_overlap_suvr_df[(tau_abeta_overlap_suvr_df['Description_abeta'].str.contains("FBB"))&~(tau_abeta_overlap_suvr_df['Description_abeta'].str.contains("AV45"))]

out_df = tau_abeta_FBB_overlap_suvr_df

# Calc A status
tau_abeta_AV45_overlap_suvr_df['Amyloid Sensitive SUVR'] = tau_abeta_AV45_overlap_suvr_df.apply(lambda row: Calc_Amyloid_Sensitive_SUVR_dkt(row), axis=1)
av45_cutoff = 1.11
fbb_cutoff = 1.08
tau_abeta_AV45_overlap_suvr_df['A+'] = tau_abeta_AV45_overlap_suvr_df.apply(lambda row: 1 if row['Amyloid Sensitive SUVR']>av45_cutoff else 0, axis=1)
print(tau_abeta_AV45_overlap_suvr_df.groupby(["A+"]).size().reset_index().rename(columns={0:'count'}))

tau_abeta_FBB_overlap_suvr_df['Amyloid Sensitive SUVR'] = tau_abeta_FBB_overlap_suvr_df.apply(lambda row: Calc_Amyloid_Sensitive_SUVR_dkt(row), axis=1)
av45_cutoff = 1.11
fbb_cutoff = 1.08
tau_abeta_FBB_overlap_suvr_df['A+'] = tau_abeta_FBB_overlap_suvr_df.apply(lambda row: 1 if row['Amyloid Sensitive SUVR']>fbb_cutoff else 0, axis=1)
print(tau_abeta_FBB_overlap_suvr_df.groupby(["A+"]).size().reset_index().rename(columns={0:'count'}))

   T+  count
0   0    689
1   1     50
   A+  count
0   0    324
1   1    200
   A+  count
0   0    135
1   1     80


In [33]:
test_ids_amyloid = []
for indx, row in out_df.iterrows():
    test_ids_amyloid.append('{}|{}'.format(row['RID'],row['Study Date_tau']))

### Calc Population Statistics

In [34]:
tau_abeta_AV45_overlap_suvr_df_diag = get_overlap_dataframe(tau_abeta_AV45_overlap_suvr_df, diag_df, df1_date_name='Study Date_tau')
tau_abeta_AV45_overlap_suvr_df_diag['diag'] = tau_abeta_AV45_overlap_suvr_df_diag.apply(lambda x: get_diag(x),axis=1)

tau_abeta_FBB_overlap_suvr_df_diag = get_overlap_dataframe(tau_abeta_FBB_overlap_suvr_df, diag_df, df1_date_name='Study Date_tau', days_delta=600)
tau_abeta_FBB_overlap_suvr_df_diag['diag'] = tau_abeta_FBB_overlap_suvr_df_diag.apply(lambda x: get_diag(x),axis=1)


In [35]:
tau_abeta_FBB_overlap_suvr_df

,Subject ID,Sex,APOE A1,APOE A2,Study Date_tau,Age,Description_tau,Imaging Protocol_tau,Image ID_tau,Study Date_abeta,...,t.diff.adas.pet.yrs_abeta,PHASE_abeta,DX_abeta,SITEID_abeta,t.diff.diagnosis.pet.yrs_abeta,Phase_abeta,apoe,T+,Amyloid Sensitive SUVR,A+
60,006_S_6209,M,3.0,3.0,2018-02-20,72.3,Brain_ADNI3_AV1451_LM (AC) Tau,Radiopharmaceutical=18F-AV1451;Mfg Model=Biogr...,966777,2018-02-27,...,NaN,NaN,CN,4.0,-0.013699,ADNI3,33,0,0.990054,0
61,006_S_6234,F,3.0,4.0,2018-03-22,70.7,Brain_ADNI3_AV1451_NEWATN1-2 (AC) Tau,Radiopharmaceutical=18F-AV1451;Mfg Model=Biogr...,980183,2018-03-06,...,NaN,NaN,CN,4.0,0.002740,ADNI3,34,0,1.046531,0
62,006_S_6243,M,3.0,3.0,2018-04-04,66.9,Brain_ADNI3_AV1451_LM (AC) Tau,Radiopharmaceutical=18F-AV1451;Mfg Model=Biogr...,981657,2018-03-14,...,NaN,NaN,MCI,4.0,-0.043836,ADNI3,33,0,0.869037,0
63,006_S_6252,F,3.0,4.0,2018-04-25,74.8,Brain_ADNI3_AV1451_LM (AC) Tau,Radiopharmaceutical=18F-AV1451;Mfg Model=Biogr...,989600,2018-04-10,...,NaN,NaN,MCI,4.0,0.063014,ADNI3,34,1,1.845195,1
64,006_S_6277,F,3.0,3.0,2018-05-07,70.9,Brain_ADNI3_AV1451_LM (AC) Tau,Radiopharmaceutical=18F-AV1451;Mfg Model=Biogr...,994547,2018-04-17,...,NaN,NaN,CN,4.0,0.043836,ADNI3,33,0,0.899692,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064,941_S_6575,M,3.0,3.0,2018-10-24,73.6,ADNI3-BRAIN Tau,Radiopharmaceutical=18F-AV1451;Mfg Model=Disco...,1064703,2018-10-30,...,NaN,NaN,CN,59.0,-0.115068,ADNI3,33,0,1.277207,1
1065,941_S_6575,M,3.0,3.0,2021-01-20,75.9,ADNI3-BRAIN Tau,Radiopharmaceutical=18F-AV1451;Mfg Model=Disco...,1403267,2020-11-25,...,NaN,NaN,CN,59.0,-0.079452,ADNI3,33,0,1.382339,1
1066,941_S_6580,F,3.0,3.0,2018-10-26,80.0,ADNI3-BRAIN Tau,Radiopharmaceutical=18F-AV1451;Mfg Model=Disco...,1065699,2018-11-28,...,NaN,NaN,CN,59.0,-0.205479,ADNI3,33,0,1.408058,1
1067,941_S_6580,F,3.0,3.0,2020-10-09,82.0,ADNI3-BRAIN Tau,Radiopharmaceutical=18F-AV1451;Mfg Model=Disco...,1348905,2020-10-20,...,NaN,NaN,CN,59.0,0.005479,ADNI3,33,0,1.485173,1


In [36]:
tau_abeta_FBB_overlap_suvr_df['RID']

60      6209
61      6234
62      6243
63      6252
64      6277
        ... 
1064    6575
1065    6575
1066    6580
1067    6580
1068    6581
Name: RID, Length: 215, dtype: int64

In [53]:
with open(os.path.join(PROJECT_BASE,'data/splits_FBB_as_AV45_multitask/dkt/split_5_ids.csv'), 'w') as f:
        out = 'RID,Study Date Tau,Study Date Abeta,Diagnosis\n'
        f.write(out)
        for indx, row in tau_abeta_FBB_overlap_suvr_df_diag.iterrows():
            out = f'{row["RID"]},{row["Study Date_tau"]},{row["Study Date_abeta"]},{row["diag"]}\n'
            f.write(out)

In [43]:
pd.set_option('display.max_columns', 1000)
tau_abeta_FBB_overlap_suvr_df_diag[['diag']]

,diag
0,CN
1,CN
2,MCI
3,MCI
4,CN
...,...
210,CN
211,CN
212,CN
213,CN


In [35]:
# Split by diagnosis and calculate mean age, sex, amyloid SUVR and tau SUVR
print(f"Train AV45 N={len(tau_abeta_AV45_overlap_suvr_df_diag)}")
print(f"Test FBB N={len(tau_abeta_FBB_overlap_suvr_df_diag)}")
print('---')
for split_df in [tau_abeta_AV45_overlap_suvr_df_diag, tau_abeta_FBB_overlap_suvr_df_diag]:
  for d in ['CN','MCI','AD']:
    temp_diag_df = split_df[split_df['diag']==d]
    f_count = sum(temp_diag_df['Sex']=='F')
    m_count = sum(temp_diag_df['Sex']=='M')
    mean_age = temp_diag_df['Age'].mean()
    std_age = temp_diag_df['Age'].std()
    mean_global_tau = temp_diag_df['tau.global.SUVR_tau'].mean()
    std_global_tau = temp_diag_df['tau.global.SUVR_tau'].std()
    mean_global_amyloid = temp_diag_df['amyloid.global.SUVR_abeta'].mean()
    std_global_amyloid = temp_diag_df['amyloid.global.SUVR_abeta'].std()
    amyloid_pos_count = sum(temp_diag_df['A+']==1)
    amyloid_neg_count = sum(temp_diag_df['A+']==0)
    tau_pos_count = sum(temp_diag_df['T+']==1)
    tau_neg_count = sum(temp_diag_df['T+']==0)
    mean_mmse = temp_diag_df['MMSE_tau'].mean()
    std_mmse = temp_diag_df['MMSE_tau'].std()
    mean_adas13 = temp_diag_df['ADAS13_tau'].mean()
    std_adas13 = temp_diag_df['ADAS13_tau'].std()
    apoe_neg_count = sum(temp_diag_df['apoe_tau']==33)
    apoe_pos_count = sum(temp_diag_df['apoe_tau']==34)+sum(temp_diag_df['apoe_tau']==43)
    print(f"For Diagnosis {d} (N={len(temp_diag_df)})\nSex (f/m): {f_count}/{m_count}\nAge: {mean_age:.2f} ± {std_age:.2f}\nGlobal Tau: {mean_global_tau:.2f} ± {std_global_tau:.2f}\nGlobal Amyloid: {mean_global_amyloid:.2f} ± {std_global_amyloid:.2f}\nAmyloid Status (+/-): {amyloid_pos_count}/{amyloid_neg_count}\nTau Status (+/-): {tau_pos_count}/{tau_neg_count}\nMMSE: {mean_mmse:.2f} ± {std_mmse:.2f}\nADAS13: {mean_adas13:.2f} ± {std_adas13:.2f}\nAPOE Status (+/-): {apoe_pos_count}/{apoe_neg_count}")
    print('---')
  print('@@@@')

Train AV45 N=502
Test FBB N=212
---
For Diagnosis CN (N=312)
Sex (f/m): 178/134
Age: 75.42 ± 7.16
Global Tau: 1.07 ± 0.09
Global Amyloid: 1.06 ± 0.20
Amyloid Status (+/-): 87/225
Tau Status (+/-): 5/307
MMSE: 29.02 ± 1.30
ADAS13: 7.64 ± 4.36
APOE Status (+/-): 94/165
---
For Diagnosis MCI (N=145)
Sex (f/m): 56/89
Age: 77.64 ± 7.28
Global Tau: 1.12 ± 0.15
Global Amyloid: 1.14 ± 0.28
Amyloid Status (+/-): 65/80
Tau Status (+/-): 9/136
MMSE: 28.03 ± 2.04
ADAS13: 13.39 ± 6.46
APOE Status (+/-): 35/81
---
For Diagnosis AD (N=45)
Sex (f/m): 19/26
Age: 78.74 ± 9.49
Global Tau: 1.27 ± 0.36
Global Amyloid: 1.36 ± 0.24
Amyloid Status (+/-): 40/5
Tau Status (+/-): 14/31
MMSE: 21.63 ± 4.94
ADAS13: 29.63 ± 10.85
APOE Status (+/-): 14/23
---
@@@@
For Diagnosis CN (N=144)
Sex (f/m): 88/56
Age: 71.18 ± 5.93
Global Tau: 1.08 ± 0.10
Global Amyloid: 1.06 ± 0.22
Amyloid Status (+/-): 44/100
Tau Status (+/-): 5/139
MMSE: 29.54 ± 0.78
ADAS13: 8.44 ± 4.89
APOE Status (+/-): 52/77
---
For Diagnosis MCI (N=46)

### Split and Save

In [36]:
all_groups = split_groups(tau_abeta_AV45_overlap_suvr_df)

105
105
105
105
104


In [37]:
def encode_apoe_one_hot(apoe):
    if apoe == '33':
        return '1,0,0,0,0,0'
    if apoe == '34':
        return '0,1,0,0,0,0'
    if apoe == '44':
        return '0,0,1,0,0,0'
    if apoe == '24':
        return '0,0,0,1,0,0'
    if apoe == '23':
        return '0,0,0,0,1,0'
    if apoe == '22':
        return '0,0,0,0,0,1'
    
def encode_suvr(row,protein):
    out = ''
    for i in dktLabels:
        out += '{},'.format(row['SUVR.DKT.ROI.idx.{}_{}'.format(i,protein)])
    return out[:-1]

def encode_suvr_fbb_as_av45(row,protein):
    out = ''
    for i in dktLabels:
        out += '{},'.format(Calc_AV45_From_FBB(row['SUVR.DKT.ROI.idx.{}_{}'.format(i,protein)]))
    return out[:-1]


def encode_m_f(sex):
    if sex == 'M':
        return '0,1'
    else:
        return '1,0'

mean_age = suvr_df[suvr_df['pet.modality']=='pet-AV45']['age'].mean()

if SAVE:
    with open(os.path.join(PROJECT_BASE,'data/splits_FBB_as_AV45_multitask/dkt/split_5_diag.csv'), 'w') as f:
        for indx, row in tau_abeta_FBB_overlap_suvr_df_diag.iterrows():
            out = '{},{},{},{},{},{},{}\n'.format(encode_m_f(row['Sex']),str(row['Age']-mean_age),encode_apoe_one_hot(row['apoe']),encode_suvr(row,'tau'),encode_suvr_fbb_as_av45(row,'abeta'),row['A+'],row['diag'])
            f.write(out)
      
    with open(os.path.join(PROJECT_BASE,'data/splits_FBB_as_AV45_multitask/dkt/split_5.csv'), 'w') as f:
        for indx, row in tau_abeta_FBB_overlap_suvr_df_diag.iterrows():
            out = '{},{},{},{},{},{}\n'.format(encode_m_f(row['Sex']),str(row['Age']-mean_age),encode_apoe_one_hot(row['apoe']),encode_suvr(row,'tau'),encode_suvr_fbb_as_av45(row,'abeta'),row['A+'])
            f.write(out)

    for s,g in enumerate(all_groups):
        with open(os.path.join(PROJECT_BASE,'data/splits_AV45_multitask/dkt/split_{}.csv'.format(s)), 'w') as f:
            for i in g:
                row = tau_abeta_AV45_overlap_suvr_df.loc[i]
                #out = '{},{},{},{},{}\n'.format(encode_m_f(row['Sex']),str(row['Age']-mean_age),encode_apoe_one_hot(row['apoe']),encode_suvr(row,'tau'),encode_suvr_fbb_as_av45(row,'abeta'))
                out = '{},{},{},{},{},{}\n'.format(encode_m_f(row['Sex']),str(row['Age']-mean_age),encode_apoe_one_hot(row['apoe']),encode_suvr(row,'tau'),encode_suvr(row,'abeta'),row['A+'])
                f.write(out)

In [68]:
with open(os.path.join(PROJECT_BASE,'data/splits_FBB_as_AV45_multitask/dkt/split_5_diags.csv'), 'w') as f:
        for indx, row in tau_abeta_FBB_overlap_suvr_df_diag.iterrows():
            out = '{}\n'.format(row['diag'])
            f.write(out)

## Create corresponding gmvolume dataset

In [42]:
# Check which volume files are on PC
t1_tau_overlap_df['RID'] = t1_tau_overlap_df.apply(lambda row: int(row['Subject ID'].split('_')[2]), axis=1)

my_files_df = pd.DataFrame(columns=['RID','Study Date'])
for filename in glob.iglob('{}/**/*AALVolume_GM_visit_masked.csv'.format(ADNI_DATA_BASE),recursive = True):
    split = filename.split('/')
    rid = split[-5].split('sub-')[1]
    ses = split[-3].split('ses-')[1]
    my_files_df = my_files_df.append({'RID':rid,'Study Date':ses}, ignore_index=True)
my_files_df['Study Date'] = pd.to_datetime(my_files_df['Study Date'])
my_files_df['RID'] = pd.to_numeric(my_files_df['RID'])

In [43]:
len(my_files_df)

6211

In [44]:
# Neurodegen Test
available_t1_tau_df = t1_tau_overlap_df.merge(my_files_df, right_on=['RID','Study Date'], left_on=['RID','Study Date_t1'])
t1_tau_FBB_df = available_t1_tau_df.merge(tau_abeta_FBB_overlap_suvr_df, on=['RID','Study Date_tau'])
print("Out of {} Amyloid/Tau files, {} have an MRI within 6 months of the Tau-PET".format(len(tau_abeta_FBB_overlap_suvr_df),len(t1_tau_FBB_df)))
out_split_df_test = t1_tau_FBB_df

# Grab SUVR values
suvr_df = pd.read_csv(os.path.join(DATA_BASE,'ADNI_Tau_Amyloid_SUVR_status.csv'))
suvr_df['RID'] = pd.to_numeric(suvr_df['RID'])
suvr_df['acq.date'] = pd.to_datetime(suvr_df['acq.date'])
t1_tau_overlap_df_copy = out_split_df_test.copy(deep=True)
t1_tau_overlap_suvr_df = t1_tau_overlap_df_copy.merge(suvr_df[suvr_df['pet.modality']=='pet-AV1451'], how='left', left_on=['RID','Study Date_tau'], right_on=['RID','acq.date'])
t1_tau_overlap_suvr_df = t1_tau_overlap_suvr_df.dropna(subset=['SUVR.Schaefer200.ROI.idx.1','apoe_y'])

len(t1_tau_overlap_suvr_df)
out_df_test=t1_tau_overlap_suvr_df.rename(columns={'apoe_y':'apoe'})

# Grab gmvol values
GM_volume_df_test = pd.DataFrame(columns=['RID','Study Date'].extend(commonLabels))
for indx, row in out_df_test.iterrows():
    rid = row['RID']
    date = row['Study Date_t1']
    filename = os.path.join(ADNI_DATA_BASE,'sub-{}'.format(str(rid).zfill(4)),"anat","ses-{}".format(str(date)[:10]),"antsCorticalThickness",'AALVolume_GM_visit_masked.csv')
    with open(filename, 'r') as f:
        lines = f.readlines()
    means = [float(l.split()[1]) for i,l in enumerate(lines) if i>0]
    labels = [int(l.split()[0]) for i,l in enumerate(lines) if i>0]
    toKeep = []
    for i,l in enumerate(labels):
        if l in commonLabels:
            toKeep.append(i)
    if len(toKeep) != len(commonLabels):
        print('skip | {} | {} | {}'.format(len(toKeep),len(commonLabels),filename))
        continue
    means = [means[i] for i in toKeep]
    newRow = {'RID':int(rid),'Study Date':date}
    for i, l in enumerate(commonLabels):
        newRow[l] = means[i]
    GM_volume_df_test = GM_volume_df_test.append(newRow, ignore_index=True)

# merge gmvol values and suvr values
out_GM_volume_df_test = out_df_test.merge(GM_volume_df_test, left_on=['RID','Study Date_t1'], right_on=['RID','Study Date'])

# Calc N status
out_GM_volume_df_test['Hippocampus Volume'] = out_GM_volume_df_test.apply(lambda row: calc_hipp_vol(row),axis=1)
cutoff = 1.6474876557377052 - 1 * 0.08779840023070665 # Hipp only
cutoff = 1.6617301245901637 - 1 * 0.08045998796511386
out_GM_volume_df_test['N+'] = out_GM_volume_df_test.apply(lambda row: 1 if row['Hippocampus Volume']<cutoff else 0, axis=1)

out_GM_volume_df_test.groupby(["N+"]).size().reset_index().rename(columns={0:'count'})


Out of 215 Amyloid/Tau files, 208 have an MRI within 6 months of the Tau-PET


,N+,count
0,0,167
1,1,41


In [45]:
# Neurodegen train
merge = available_t1_tau_df.merge(tau_abeta_FBB_overlap_suvr_df, on=['RID','Study Date_tau'], how='outer', indicator='source')
t1_tau_noFBB_df = merge[merge.source.eq('left_only')].drop('source', axis=1)

print("Out of {} Tau files, {} have an MRI within 6 months of the Tau-PET and are not FBB".format(len(available_t1_tau_df),len(t1_tau_noFBB_df)))
out_split_df_train = t1_tau_noFBB_df

# Grab SUVR values
suvr_df = pd.read_csv(os.path.join(DATA_BASE,'ADNI_Tau_Amyloid_SUVR_status.csv'))
suvr_df['RID'] = pd.to_numeric(suvr_df['RID'])
suvr_df['acq.date'] = pd.to_datetime(suvr_df['acq.date'])
t1_tau_overlap_df_copy = out_split_df_train.copy(deep=True)
t1_tau_overlap_suvr_df = t1_tau_overlap_df_copy.merge(suvr_df[suvr_df['pet.modality']=='pet-AV1451'], how='left', left_on=['RID','Study Date_tau'], right_on=['RID','acq.date'])
t1_tau_overlap_suvr_df = t1_tau_overlap_suvr_df.dropna(subset=['SUVR.Schaefer200.ROI.idx.1','apoe_y'])

len(t1_tau_overlap_suvr_df)
out_df_train=t1_tau_overlap_suvr_df.rename(columns={'apoe_y':'apoe'})

# Grab gmvol values
GM_volume_df_train = pd.DataFrame(columns=['RID','Study Date'].extend(commonLabels))
for indx, row in out_df_train.iterrows():
    rid = row['RID']
    date = row['Study Date_t1']
    filename = os.path.join(ADNI_DATA_BASE,'sub-{}'.format(str(rid).zfill(4)),"anat","ses-{}".format(str(date)[:10]),"antsCorticalThickness",'AALVolume_GM_visit_masked.csv')
    with open(filename, 'r') as f:
        lines = f.readlines()
    means = [float(l.split()[1]) for i,l in enumerate(lines) if i>0]
    labels = [int(l.split()[0]) for i,l in enumerate(lines) if i>0]
    toKeep = []
    for i,l in enumerate(labels):
        if l in commonLabels:
            toKeep.append(i)
    if len(toKeep) != len(commonLabels):
        print('skip | {} | {} | {}'.format(len(toKeep),len(commonLabels),filename))
        continue
    means = [means[i] for i in toKeep]
    newRow = {'RID':int(rid),'Study Date':date}
    for i, l in enumerate(commonLabels):
        newRow[l] = means[i]
    GM_volume_df_train = GM_volume_df_train.append(newRow, ignore_index=True)
    
# merge gmvol values and suvr values
out_GM_volume_df_train = out_df_train.merge(GM_volume_df_train, left_on=['RID','Study Date_t1'], right_on=['RID','Study Date'])

# Calc N status
out_GM_volume_df_train['Hippocampus Volume'] = out_GM_volume_df_train.apply(lambda row: calc_hipp_vol(row),axis=1)
cutoff = 1.6474876557377052 - 1 * 0.08779840023070665 # Hipp only
cutoff = 1.6617301245901637 - 1 * 0.08045998796511386
out_GM_volume_df_train['N+'] = out_GM_volume_df_train.apply(lambda row: 1 if row['Hippocampus Volume']<cutoff else 0, axis=1)

out_GM_volume_df_train.groupby(["N+"]).size().reset_index().rename(columns={0:'count'})

Out of 1154 Tau files, 946 have an MRI within 6 months of the Tau-PET and are not FBB
skip | 0 | 98 | /home/guests/projects/adni/data_bids_processed/sub-4400/anat/ses-2017-09-26/antsCorticalThickness/AALVolume_GM_visit_masked.csv
skip | 97 | 98 | /home/guests/projects/adni/data_bids_processed/sub-4197/anat/ses-2015-09-21/antsCorticalThickness/AALVolume_GM_visit_masked.csv


,N+,count
0,0,499
1,1,200


In [46]:
print(len(out_GM_volume_df_train))
print(len(out_GM_volume_df_test))

699
208


### Calc Population Statistics

In [47]:
out_GM_volume_df_train_diag = get_overlap_dataframe(out_GM_volume_df_train, diag_df, df1_id_name='Subject ID_x', df1_date_name='Study Date_tau')
out_GM_volume_df_train_diag['diag'] = out_GM_volume_df_train_diag.apply(lambda x: get_diag(x),axis=1)
out_GM_volume_df_train_diag['T+'] = out_GM_volume_df_train_diag.apply(lambda row: 1 if row['tau.global.SUVR']>1.3 else 0, axis=1)

out_GM_volume_df_test_diag = get_overlap_dataframe(out_GM_volume_df_test, diag_df, df1_id_name='Subject ID_x', df1_date_name='Study Date_tau', days_delta=600)
out_GM_volume_df_test_diag['diag'] = out_GM_volume_df_test_diag.apply(lambda x: get_diag(x),axis=1)
out_GM_volume_df_test_diag['T+'] = out_GM_volume_df_test_diag.apply(lambda row: 1 if row['tau.global.SUVR']>1.3 else 0, axis=1)

all_subjects_df = pd.concat([out_GM_volume_df_train,out_GM_volume_df_test])
all_subjects_diag_df = get_overlap_dataframe(all_subjects_df, diag_df, df1_id_name='Subject ID_x', df1_date_name='Study Date_tau')
all_subjects_diag_df['diag'] = all_subjects_diag_df.apply(lambda x: get_diag(x),axis=1)



In [62]:
print(len(out_GM_volume_df_test))
print(len(out_GM_volume_df_test_diag))

207
207


In [45]:
# Split by diagnosis and calculate mean age, sex, amyloid SUVR and tau SUVR
print(f"Train N={len(out_GM_volume_df_train_diag)}")
print(f"Test N={len(out_GM_volume_df_test_diag)}")
print('---')
for split_df in [out_GM_volume_df_train_diag, out_GM_volume_df_test_diag]:
    for d in ['CN','MCI','AD']:
        temp_diag_df = split_df[split_df['diag']==d]
        f_count = sum(temp_diag_df['Sex_x']=='F')
        m_count = sum(temp_diag_df['Sex_x']=='M')
        mean_age = temp_diag_df['Age_x'].mean()
        std_age = temp_diag_df['Age_x'].std()
        mean_global_tau = temp_diag_df['tau.global.SUVR'].mean()
        std_global_tau = temp_diag_df['tau.global.SUVR'].std()
        tau_pos_count = sum(temp_diag_df['T+']==1)
        tau_neg_count = sum(temp_diag_df['T+']==0)
        neurodegen_pos_count = sum(temp_diag_df['N+']==1)
        neurodegen_neg_count = sum(temp_diag_df['N+']==0)
        mean_mmse = temp_diag_df['MMSE'].mean()
        std_mmse = temp_diag_df['MMSE'].std()
        mean_adas13 = temp_diag_df['ADAS13'].mean()
        std_adas13 = temp_diag_df['ADAS13'].std()
        apoe_neg_count = sum(temp_diag_df['apoe']==33)
        apoe_pos_count = sum(temp_diag_df['apoe']==34)+sum(temp_diag_df['apoe']==43)
        print(f"For Diagnosis {d} (N={len(temp_diag_df)})\nSex (f/m): {f_count}/{m_count}\nAge: {mean_age:.2f} ± {std_age:.2f}\nGlobal Tau: {mean_global_tau:.2f} ± {std_global_tau:.2f}\nNeurodegen Status (+/-): {neurodegen_pos_count}/{neurodegen_neg_count}\nTau Status (+/-): {tau_pos_count}/{tau_neg_count}\nMMSE: {mean_mmse:.2f} ± {std_mmse:.2f}\nADAS13: {mean_adas13:.2f} ± {std_adas13:.2f}\nAPOE Status (+/-): {apoe_pos_count}/{apoe_neg_count}")
        print('---')
        print('@@@@')

Train N=671
Test N=207
---
For Diagnosis CN (N=404)
Sex (f/m): 236/168
Age: 74.98 ± 6.96
Global Tau: 1.08 ± 0.09
Neurodegen Status (+/-): 64/340
Tau Status (+/-): 8/396
MMSE: 29.09 ± 1.17
ADAS13: 7.63 ± 4.43
APOE Status (+/-): 131/215
---
@@@@
For Diagnosis MCI (N=201)
Sex (f/m): 78/123
Age: 76.95 ± 7.06
Global Tau: 1.13 ± 0.16
Neurodegen Status (+/-): 48/153
Tau Status (+/-): 18/183
MMSE: 27.93 ± 2.00
ADAS13: 13.32 ± 6.04
APOE Status (+/-): 56/105
---
@@@@
For Diagnosis AD (N=66)
Sex (f/m): 27/39
Age: 76.97 ± 9.29
Global Tau: 1.35 ± 0.42
Neurodegen Status (+/-): 36/30
Tau Status (+/-): 25/41
MMSE: 21.71 ± 4.85
ADAS13: 29.93 ± 10.40
APOE Status (+/-): 21/29
---
@@@@
For Diagnosis CN (N=139)
Sex (f/m): 87/52
Age: 71.05 ± 5.94
Global Tau: 1.08 ± 0.11
Neurodegen Status (+/-): 14/125
Tau Status (+/-): 5/134
MMSE: 29.54 ± 0.78
ADAS13: 8.44 ± 4.89
APOE Status (+/-): 50/74
---
@@@@
For Diagnosis MCI (N=46)
Sex (f/m): 20/26
Age: 71.14 ± 8.05
Global Tau: 1.13 ± 0.19
Neurodegen Status (+/-): 3/4

### Split and Save

In [46]:
all_groups = split_groups(out_GM_volume_df_train, num_splits=5)

140
140
140
140
139


In [57]:
out_GM_volume_df_test = out_GM_volume_df_test.drop_duplicates(subset=['RID','Study Date_tau'])
test_ids_gmdensity = []
for indx, row in out_GM_volume_df_test.iterrows():
    test_ids_gmdensity.append('{}|{}'.format(row['RID'],row['Study Date_tau']))

In [48]:
def encode_apoe_one_hot(apoe):
    apoe = str(int(apoe))
    if apoe == '33':
        return '1,0,0,0,0,0'
    if apoe == '34':
        return '0,1,0,0,0,0'
    if apoe == '44':
        return '0,0,1,0,0,0'
    if apoe == '24':
        return '0,0,0,1,0,0'
    if apoe == '23':
        return '0,0,0,0,1,0'
    if apoe == '22':
        return '0,0,0,0,0,1'
    
def encode_suvr_dkt(row,protein):
    out = ''
    for i in dktLabels:
        out += '{},'.format(row['SUVR.DKT.ROI.idx.{}'.format(i,protein)])
    return out[:-1]

def encode_suvr_schaefer(row,protein):
    out = ''
    for i in range(1,201):
        out += '{},'.format(row['SUVR.Schaefer200.ROI.idx.{}'.format(i)])
    return out[:-1]

def encode_vol(row):
    out = ''
    for l in commonLabels:
        out += '{},'.format(row[l])
    return out[:-1]

def encode_m_f(sex):
    if sex == 'M':
        return '0,1'
    else:
        return '1,0'

mean_age = out_GM_volume_df_train['Age_x'].mean()

folder = 'splits_volume_ATN_schaefer_multitask'
enc = encode_suvr_schaefer
cnt = 0

if SAVE:
    for s,g in enumerate(all_groups):
        with open(os.path.join(PROJECT_BASE,'data/{}/split_{}.csv'.format(folder,s)), 'w') as f:
            for i in g:
                row = out_GM_volume_df_train.loc[i]
                out = '{},{},{},{},{},{}\n'.format(encode_m_f(row['Sex_x']),str(row['Age_x']-mean_age),encode_apoe_one_hot(row['apoe']),enc(row,'tau'),encode_vol(row),row['N+'])
                if 'na' not in out:
                    f.write(out)
                else:
                    cnt = cnt+1
                    print(cnt)
    s=5
    with open(os.path.join(PROJECT_BASE,'data/{}/split_{}.csv'.format(folder,s)), 'w') as f:
        for indx, row in out_GM_volume_df_test.iterrows():
            row = out_GM_volume_df_test.loc[indx]
            out = '{},{},{},{},{},{}\n'.format(encode_m_f(row['Sex_x']),str(row['Age_x']-mean_age),encode_apoe_one_hot(row['apoe']),enc(row,'tau'),encode_vol(row),row['N+'])
            f.write(out)

    with open(os.path.join(PROJECT_BASE,'data/{}/split_{}_diag.csv'.format(folder,s)), 'w') as f:
        for indx, row in out_GM_volume_df_test_diag.iterrows():
            row = out_GM_volume_df_test_diag.loc[indx]
            out = '{},{},{},{},{},{},{}\n'.format(encode_m_f(row['Sex_x']),str(row['Age_x']-mean_age),encode_apoe_one_hot(row['apoe']),enc(row,'tau'),encode_vol(row),row['N+'],row['diag'])
            f.write(out)
    

In [63]:
def encode_apoe_one_hot(apoe):
    apoe = str(int(apoe))
    if apoe == '33':
        return '1,0,0,0,0,0'
    if apoe == '34':
        return '0,1,0,0,0,0'
    if apoe == '44':
        return '0,0,1,0,0,0'
    if apoe == '24':
        return '0,0,0,1,0,0'
    if apoe == '23':
        return '0,0,0,0,1,0'
    if apoe == '22':
        return '0,0,0,0,0,1'
    
def encode_suvr_dkt(row,protein):
    out = ''
    for i in dktLabels:
        out += '{},'.format(row['SUVR.DKT.ROI.idx.{}'.format(i,protein)])
    return out[:-1]

def encode_suvr_schaefer(row,protein):
    out = ''
    for i in range(1,201):
        out += '{},'.format(row['SUVR.Schaefer200.ROI.idx.{}'.format(i)])
    return out[:-1]

def encode_vol(row):
    out = ''
    for l in commonLabels:
        out += '{},'.format(row[l])
    return out[:-1]

def encode_m_f(sex):
    if sex == 'M':
        return '0,1'
    else:
        return '1,0'

mean_age = out_GM_volume_df_train['Age_x'].mean()

folder = 'splits_volume_ATN_schaefer_multitask'
enc = encode_suvr_schaefer
cnt = 0
s=5
with open(os.path.join(PROJECT_BASE,'data/{}/split_{}_diag.csv'.format(folder,s)), 'w') as f:
        for indx, row in out_GM_volume_df_test_diag.iterrows():
            row = out_GM_volume_df_test_diag.loc[indx]
            out = '{},{},{},{},{},{},{}\n'.format(encode_m_f(row['Sex_x']),str(row['Age_x']-mean_age),encode_apoe_one_hot(row['apoe']),enc(row,'tau'),encode_vol(row),row['N+'],row['diag'])
            f.write(out)

In [67]:
with open(os.path.join(PROJECT_BASE,'data/{}/split_{}_diags.csv'.format(folder,s)), 'w') as f:
  for indx, row in out_GM_volume_df_test_diag.iterrows():
    f.write('{}\n'.format(row['diag']))

In [52]:
with open(os.path.join(PROJECT_BASE,'data/splits_volume_ATN_schaefer_multitask/split_5_ids.csv'), 'w') as f:
        out = 'RID,Study Date Tau,Study Date T1,Diagnosis\n'
        f.write(out)
        for indx, row in out_GM_volume_df_test_diag.iterrows():
            out = f'{row["RID"]},{row["Study Date_tau"]},{row["Study Date_t1"]},{row["diag"]}\n'
            f.write(out)

In [49]:
# Check overlap of test sets
print('Amyloid test set size is {}, and gm density is {}'.format(len(test_ids_amyloid),len(test_ids_gmdensity)))
test_ids_amyloid_overlap_gmdensity_indices_str = [str(i) for i in range(len(test_ids_amyloid)) if test_ids_amyloid[i] in test_ids_gmdensity]
test_ids_amyloid_overlap_gmdensity_indices = [i for i in range(len(test_ids_amyloid)) if test_ids_amyloid[i] in test_ids_gmdensity]
print(len(test_ids_amyloid_overlap_gmdensity_indices))
print(','.join(test_ids_amyloid_overlap_gmdensity_indices_str))
if False:
    for i in range(len(test_ids_amyloid_overlap_gmdensity_indices)):
        #print(test_ids_amyloid_overlap_gmdensity_indices[i])
        print('Amyloid: {} | GMDensity: {}'.format(test_ids_amyloid[test_ids_amyloid_overlap_gmdensity_indices[i]],test_ids_gmdensity[i]))


Amyloid test set size is 215, and gm density is 207
207
0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,54,55,56,57,58,59,60,61,62,63,64,65,66,67,69,70,71,72,73,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,212,213,214


## Calc mean hippocampus volume A-/T-/CN

In [91]:
GM_volume_df = pd.DataFrame(columns=['RID','Study Date'].extend(commonLabels))
for indx, row in my_files_df.iterrows():
    rid = row['RID']
    date = row['Study Date']
    filename = os.path.join(ADNI_DATA_BASE,'sub-{}'.format(str(rid).zfill(4)),"anat","ses-{}".format(str(date)[:10]),"antsCorticalThickness",'AALVolume_GM_visit_masked.csv')
    with open(filename, 'r') as f:
        lines = f.readlines()
    means = [float(l.split()[1]) for i,l in enumerate(lines) if i>0]
    labels = [int(l.split()[0]) for i,l in enumerate(lines) if i>0]
    toKeep = []
    for i,l in enumerate(labels):
        if l in commonLabels:
            toKeep.append(i)
    if len(toKeep) != len(commonLabels):
        print('skip | {} | {} | {}'.format(len(toKeep),len(commonLabels),filename))
        continue
    means = [means[i] for i in toKeep]
    newRow = {'RID':int(rid),'Study Date':date}
    for i, l in enumerate(commonLabels):
        newRow[l] = means[i]
    GM_volume_df = GM_volume_df.append(newRow, ignore_index=True)

skip | 0 | 98 | /home/guests/projects/adni/data_bids_processed/sub-4400/anat/ses-2017-09-26/antsCorticalThickness/AALVolume_GM_visit_masked.csv
skip | 0 | 98 | /home/guests/projects/adni/data_bids_processed/sub-6297/anat/ses-2019-05-28/antsCorticalThickness/AALVolume_GM_visit_masked.csv
skip | 89 | 98 | /home/guests/projects/adni/data_bids_processed/sub-4827/anat/ses-2013-01-30/antsCorticalThickness/AALVolume_GM_visit_masked.csv
skip | 0 | 98 | /home/guests/projects/adni/data_bids_processed/sub-6519/anat/ses-2018-07-31/antsCorticalThickness/AALVolume_GM_visit_masked.csv
skip | 0 | 98 | /home/guests/projects/adni/data_bids_processed/sub-6519/anat/ses-2020-10-01/antsCorticalThickness/AALVolume_GM_visit_masked.csv
skip | 0 | 98 | /home/guests/projects/adni/data_bids_processed/sub-2392/anat/ses-2012-08-17/antsCorticalThickness/AALVolume_GM_visit_masked.csv
skip | 97 | 98 | /home/guests/projects/adni/data_bids_processed/sub-4197/anat/ses-2015-09-21/antsCorticalThickness/AALVolume_GM_visit_m

In [94]:
GM_density_tau_amyloid_df = get_overlap_dataframe(GM_volume_df,tau_abeta_overlap_suvr_df,df2_name='diag',df1_name='vol',df1_id_name='RID',df2_id_name='RID',df2_date_name='Study Date_tau')
GM_density_tau_amyloid_suvr_diag_df = get_overlap_dataframe(GM_density_tau_amyloid_df,diag_df,df2_name='diag',df1_name='vol',df1_id_name='RID',df2_id_name='RID',df1_date_name='Study Date_tau')
GM_density_tau_amyloid_suvr_diag_df['Diag'] = GM_density_tau_amyloid_suvr_diag_df.apply(lambda row: get_diag(row),axis=1)

In [95]:
abeta_neg_tau_neg_cn_df = GM_density_tau_amyloid_suvr_diag_df[(GM_density_tau_amyloid_suvr_diag_df['tau.global.SUVR_tau']<1.3)&(GM_density_tau_amyloid_suvr_diag_df['amyloid.global.SUVR.status_abeta']=='Ab.neg')&(GM_density_tau_amyloid_suvr_diag_df['Diag']=='CN')]
abeta_neg_tau_neg_cn_df['Hippocampus Volume'] = abeta_neg_tau_neg_cn_df.apply(lambda row: calc_hipp_vol(row),axis=1)
mean_neg_GM_Vol = abeta_neg_tau_neg_cn_df['Hippocampus Volume'].mean()
std_neg_GM_Vol = abeta_neg_tau_neg_cn_df['Hippocampus Volume'].std()
print('Of A-/T- subjects, the mean total GM volume was {} with a std of {}'.format(mean_neg_GM_Vol, std_neg_GM_Vol))

Of A-/T- subjects, the mean total GM volume was 1.6617301245901637 with a std of 0.08045998796511386


## Train Logistic Regression and Gradient Boosting

In [98]:
# Define Dataset
def sex_to_OH(row, key):
    if row[key] == 'M':
        return 1
    else:
        return 0

out_GM_volume_df_train['Sex OH'] = out_GM_volume_df_train.apply(lambda row: sex_to_OH(row, "Sex_x"), axis=1)
out_GM_volume_df_test['Sex OH'] = out_GM_volume_df_test.apply(lambda row: sex_to_OH(row, "Sex_x"), axis=1)
tau_abeta_AV45_overlap_suvr_df['Sex OH'] = tau_abeta_AV45_overlap_suvr_df.apply(lambda row: sex_to_OH(row, 'Sex'), axis=1)
tau_abeta_FBB_overlap_suvr_df['Sex OH'] = tau_abeta_FBB_overlap_suvr_df.apply(lambda row: sex_to_OH(row, "Sex"), axis=1)


# NEURODEGENERATION
X_n = out_GM_volume_df_train[['tau.global.SUVR_tau','Age_x','Sex OH','APOE A1_x','APOE A2_x']].values

keys = ['SUVR.Schaefer200.ROI.idx.{}'.format(i) for i in range(1,201)]
keys.extend(['Age_x','Sex OH','APOE A1_x','APOE A2_x'])
X_n = out_GM_volume_df_train[keys].values

y_n = out_GM_volume_df_train[['N+']].values.flatten()

# AMYLOID
X_a = tau_abeta_AV45_overlap_suvr_df[['tau.global.SUVR_tau','Age','Sex OH','APOE A1','APOE A2']].values

keys = ['SUVR.DKT.ROI.idx.{}_tau'.format(i) for i in dktLabels]
keys.extend(['Age','Sex OH','APOE A1','APOE A2'])
X_a = tau_abeta_AV45_overlap_suvr_df[keys].values

y_a = tau_abeta_AV45_overlap_suvr_df[['A+']].values.flatten()

### Hyperparameter Searching

In [51]:
# XGBOOST sucks
# Hyperparameter search for best max_depth
for i in range(1,6):
    gbc = XGBRegressor(random_state=1995, max_depth=i,objective="binary:logistic", verbosity=0)
    cv_results = cross_validate(gbc, X_n, y_n, cv=5, scoring='roc_auc')
    mean_auc = np.mean(cv_results['test_score'])
    print('Mean AUC for max_depth={} is {}'.format(i,mean_auc))

Mean AUC for max_depth=1 is 0.8277783304236775
Mean AUC for max_depth=2 is 0.8198885080039849
Mean AUC for max_depth=3 is 0.8136749029575661
Mean AUC for max_depth=4 is 0.8173401843532867
Mean AUC for max_depth=5 is 0.817958159321722


In [52]:
# Hyperparameter search for best max_depth
for i in range(1,6):
    gbc = GradientBoostingClassifier(random_state=1995, max_depth=i)
    cv_results = cross_validate(gbc, X_n, y_n, cv=5, scoring='roc_auc')
    mean_auc = np.mean(cv_results['test_score'])
    print('Mean AUC for max_depth={} is {}'.format(i,mean_auc))

Mean AUC for max_depth=1 is 0.8319336033836441
Mean AUC for max_depth=2 is 0.8231135526618658
Mean AUC for max_depth=3 is 0.8269927342628215
Mean AUC for max_depth=4 is 0.8140701726623043
Mean AUC for max_depth=5 is 0.8141475158385673


In [53]:
# Hyperparameter search for best number of estimators
for i in [50,100,150,200,250]:
    gbc = GradientBoostingClassifier(random_state=1995, max_depth=1, n_estimators=i)
    cv_results = cross_validate(gbc, X_n, y_n, cv=5, scoring='roc_auc')
    mean_auc = np.mean(cv_results['test_score'])
    print('Mean AUC for n_esimators={} is {}'.format(i,mean_auc))

Mean AUC for n_esimators=50 is 0.8184692199505662
Mean AUC for n_esimators=100 is 0.8319336033836441
Mean AUC for n_esimators=150 is 0.8345224247257021
Mean AUC for n_esimators=200 is 0.8337323288996306
Mean AUC for n_esimators=250 is 0.8331150049872305


In [54]:
# Hyperparameter search for best max_depth
for i in range(1,6):
    gbc = GradientBoostingClassifier(random_state=1995, max_depth=i)
    cv_results = cross_validate(gbc, X_a, y_a, cv=5, scoring='roc_auc')
    mean_auc = np.mean(cv_results['test_score'])
    print('Mean AUC for max_depth={} is {}'.format(i,mean_auc))

Mean AUC for max_depth=1 is 0.7862319711538461
Mean AUC for max_depth=2 is 0.7919146634615384
Mean AUC for max_depth=3 is 0.7853365384615385
Mean AUC for max_depth=4 is 0.7817848557692308
Mean AUC for max_depth=5 is 0.7816334134615385


In [55]:
# Hyperparameter search for best max_depth
for i in [50,100,150,200,250,300]:
    gbc = GradientBoostingClassifier(random_state=1995, max_depth=2, n_estimators=i)
    cv_results = cross_validate(gbc, X_a, y_a, cv=5, scoring='roc_auc')
    mean_auc = np.mean(cv_results['test_score'])
    print('Mean AUC for n_estimators={} is {}'.format(i,mean_auc))

Mean AUC for n_estimators=50 is 0.78153125
Mean AUC for n_estimators=100 is 0.7919146634615384
Mean AUC for n_estimators=150 is 0.7900120192307692
Mean AUC for n_estimators=200 is 0.7943557692307693
Mean AUC for n_estimators=250 is 0.795890625
Mean AUC for n_estimators=300 is 0.7952127403846154


### Train and Evaluate

In [56]:
# Run this for multiple runs over different seeds for CI calc

orig_seed = 1995

all_vars = ["auc_a_lr_test","f1_a_lr_test","precision_a_lr_test","recall_a_lr_test","auc_a_gb_test","f1_a_gb_test","precision_a_gb_test","recall_a_gb_test","auc_n_lr_test","f1_n_lr_test","precision_n_lr_test","recall_n_lr_test","auc_n_gb_test","f1_n_gb_test","precision_n_gb_test","recall_n_gb_test","acc_atn_lr","f1_atn_lr","prec_atn_lr","recall_atn_lr","acc_atn_gb","f1_atn_gb","prec_atn_gb","recall_atn_gb"]
all_scores = {}
for v in all_vars:
    all_scores[v] = []

for seed in range(10,11):
    # AMYLOID
    
    # Hyperparameter search for best cutoff for F1
    best_cutoff_scorer = metrics.make_scorer(best_cutoff, needs_threshold=True)
    gbc = GradientBoostingClassifier(random_state=seed, max_depth=2, n_estimators=250)
    cv_results = cross_validate(gbc, X_a, y_a, cv=5, scoring=best_cutoff_scorer)
    mean_cutoff_gb_a = np.mean(cv_results['test_score'])
    #print('Optimal cutoff for GBC is {}'.format(mean_cutoff_gb_a))

    best_cutoff_scorer = metrics.make_scorer(best_cutoff, needs_threshold=True)
    lr = LogisticRegression(random_state=seed, max_iter=1000)
    cv_results = cross_validate(lr, X_a, y_a, cv=5, scoring=best_cutoff_scorer)
    mean_cutoff_lr_a = np.mean(cv_results['test_score'])
    #print('Optimal cutoff for LR is {}'.format(mean_cutoff_lr_a))
    
    # Evaluate Amyloid
    out_abeta_df_test = tau_abeta_FBB_overlap_suvr_df.iloc[test_ids_amyloid_overlap_gmdensity_indices]

    keys = ['SUVR.DKT.ROI.idx.{}_tau'.format(i) for i in dktLabels]
    keys.extend(['Age','Sex OH','APOE A1','APOE A2'])
    X_a_test = out_abeta_df_test[keys].values
    y_a_test = out_abeta_df_test[['A+']].values.flatten()
    
    # Train final model
    clf_lr_a = LogisticRegression(random_state=seed,max_iter=1000).fit(X_a,y_a)
    clf_gb_a = GradientBoostingClassifier(random_state=seed, max_depth=1, n_estimators=150).fit(X_a,y_a)

    #print('Performance LR:')
    y_hat_a_lr_test, auc_a_lr_test, f1_a_lr_test, precision_a_lr_test, recall_a_lr_test = evaluate_performance(clf_lr_a, X_a, y_a, X_a_test, y_a_test, mean_cutoff_lr_a, _print=False)
    #print('\n\nPerformance GB:')
    y_hat_a_gb_test, auc_a_gb_test, f1_a_gb_test, precision_a_gb_test, recall_a_gb_test = evaluate_performance(clf_gb_a, X_a, y_a, X_a_test, y_a_test, mean_cutoff_gb_a, _print=False)
    
    
    
    # NEURODEGENERATION
    
    # Hyperparameter search for best cutoff for F1
    best_cutoff_scorer = metrics.make_scorer(best_cutoff, needs_threshold=True)
    gbc = GradientBoostingClassifier(random_state=seed, max_depth=1, n_estimators=150)
    cv_results = cross_validate(gbc, X_n, y_n, cv=5, scoring=best_cutoff_scorer)
    mean_cutoff_gb_n = np.mean(cv_results['test_score'])
    #print('Optimal cutoff for GBC is {}'.format(mean_cutoff_gb_n))

    best_cutoff_scorer = metrics.make_scorer(best_cutoff, needs_threshold=True)
    lr = LogisticRegression(random_state=seed, max_iter=1000)
    cv_results = cross_validate(lr, X_n, y_n, cv=5, scoring=best_cutoff_scorer)
    mean_cutoff_lr_n = np.mean(cv_results['test_score'])
    #print('Optimal cutoff for LR is {}'.format(mean_cutoff_lr_n))
    
    # Train final model
    clf_lr_n = LogisticRegression(random_state=seed,max_iter=1000).fit(X_n,y_n)
    clf_gb_n = GradientBoostingClassifier(random_state=seed, max_depth=1, n_estimators=150).fit(X_n,y_n)
    
    # Evaluate Neurodegeneration
    keys = ['SUVR.Schaefer200.ROI.idx.{}'.format(i) for i in range(1,201)]
    keys.extend(['Age_x','Sex OH','APOE A1_x','APOE A2_x'])
    #X_n_test = out_GM_volume_df_test[['tau.global.SUVR','Age_x','Sex OH','APOE A1_x','APOE A2_x']].values
    X_n_test = out_GM_volume_df_test[keys].values
    y_n_test = out_GM_volume_df_test[['N+']].values.flatten()

    #print('Performance LR:')
    y_hat_n_lr_test, auc_n_lr_test, f1_n_lr_test, precision_n_lr_test, recall_n_lr_test = evaluate_performance(clf_lr_n, X_n, y_n, X_n_test, y_n_test, mean_cutoff_lr_n, _print=False)
    #print('\n\nPerformance GB:')
    y_hat_n_gb_test, auc_n_gb_test, f1_n_gb_test, precision_n_gb_test, recall_n_gb_test = evaluate_performance(clf_gb_n, X_n, y_n, X_n_test, y_n_test, mean_cutoff_gb_n, _print=False)
    
    
    
    # ATN CLASSIFICATION
    
    # Logistic Regression
    gt = classify_atn(y_n_test,y_a_test)
    pred = classify_atn(y_hat_n_lr_test,y_hat_a_lr_test)
    acc_atn_lr = metrics.accuracy_score(gt,pred)
    f1_atn_lr = metrics.f1_score(gt,pred,average='macro')
    prec_atn_lr = metrics.precision_score(gt,pred,average='macro')
    recall_atn_lr = metrics.recall_score(gt,pred,average='macro')
    #print('For ATN classification with LR, accuracy was {} and F1 was {} (Prec: {} | Rec: {})'.format(acc_atn_lr,f1_atn_lr,prec_atn_lr,recall_atn_lr))
    
    # Gradient Boosting
    gt = classify_atn(y_n_test,y_a_test)
    pred = classify_atn(y_hat_n_gb_test,y_hat_a_gb_test)
    acc_atn_gb = metrics.accuracy_score(gt,pred)
    f1_atn_gb = metrics.f1_score(gt,pred,average='macro')
    prec_atn_gb = metrics.precision_score(gt,pred,average='macro')
    recall_atn_gb = metrics.recall_score(gt,pred,average='macro')
    #print('For ATN classification with GB, accuracy was {} and F1 was {} (Prec: {} | Rec: {})'.format(acc_atn_gb,f1_atn_gb,prec_atn_gb,recall_atn_gb))
    
    for v in all_vars:
        all_scores[v].append(eval(v))
    

Prediction time was 9.441375732421875e-05
Prediction time was 0.0006780624389648438
Prediction time was 0.0001811981201171875
Prediction time was 0.0009045600891113281


In [57]:
for v in all_vars:
    calc_mean_std(v, all_scores)

auc_a_lr_test       : 0.87 ± 0.00
f1_a_lr_test        : 0.69 ± 0.00
precision_a_lr_test : 0.69 ± 0.00
recall_a_lr_test    : 0.70 ± 0.00
auc_a_gb_test       : 0.85 ± 0.00
f1_a_gb_test        : 0.70 ± 0.00
precision_a_gb_test : 0.77 ± 0.00
recall_a_gb_test    : 0.63 ± 0.00
auc_n_lr_test       : 0.82 ± 0.00
f1_n_lr_test        : 0.54 ± 0.00
precision_n_lr_test : 0.49 ± 0.00
recall_n_lr_test    : 0.60 ± 0.00
auc_n_gb_test       : 0.81 ± 0.00
f1_n_gb_test        : 0.51 ± 0.00
precision_n_gb_test : 0.52 ± 0.00
recall_n_gb_test    : 0.50 ± 0.00
acc_atn_lr          : 0.69 ± 0.00
f1_atn_lr           : 0.48 ± 0.00
prec_atn_lr         : 0.46 ± 0.00
recall_atn_lr       : 0.55 ± 0.00
acc_atn_gb          : 0.70 ± 0.00
f1_atn_gb           : 0.52 ± 0.00
prec_atn_gb         : 0.54 ± 0.00
recall_atn_gb       : 0.53 ± 0.00


In [58]:
for v in all_vars:
    calc_CI(v, all_scores)

auc_a_lr_test       : 0.87 [0.87, 0.87]
f1_a_lr_test        : 0.69 [0.69, 0.69]
precision_a_lr_test : 0.69 [0.69, 0.69]
recall_a_lr_test    : 0.70 [0.70, 0.70]
auc_a_gb_test       : 0.85 [0.85, 0.85]
f1_a_gb_test        : 0.70 [0.70, 0.70]
precision_a_gb_test : 0.77 [0.77, 0.77]
recall_a_gb_test    : 0.63 [0.63, 0.63]
auc_n_lr_test       : 0.82 [0.82, 0.82]
f1_n_lr_test        : 0.54 [0.54, 0.54]
precision_n_lr_test : 0.49 [0.49, 0.49]
recall_n_lr_test    : 0.60 [0.60, 0.60]
auc_n_gb_test       : 0.81 [0.81, 0.81]
f1_n_gb_test        : 0.51 [0.51, 0.51]
precision_n_gb_test : 0.52 [0.52, 0.52]
recall_n_gb_test    : 0.50 [0.50, 0.50]
acc_atn_lr          : 0.69 [0.69, 0.69]
f1_atn_lr           : 0.48 [0.48, 0.48]
prec_atn_lr         : 0.46 [0.46, 0.46]
recall_atn_lr       : 0.55 [0.55, 0.55]
acc_atn_gb          : 0.70 [0.70, 0.70]
f1_atn_gb           : 0.52 [0.52, 0.52]
prec_atn_gb         : 0.54 [0.54, 0.54]
recall_atn_gb       : 0.53 [0.53, 0.53]


In [99]:
# AMYLOID
# Hyperparameter search for best cutoff for F1
best_cutoff_scorer = metrics.make_scorer(best_cutoff, needs_threshold=True)
gbc = GradientBoostingClassifier(random_state=1995, max_depth=2, n_estimators=250)
cv_results = cross_validate(gbc, X_a, y_a, cv=5, scoring=best_cutoff_scorer)
mean_cutoff_gb_a = np.mean(cv_results['test_score'])
print('Optimal cutoff for GBC is {}'.format(mean_cutoff_gb_a))

best_cutoff_scorer = metrics.make_scorer(best_cutoff, needs_threshold=True)
lr = LogisticRegression(random_state=1995, max_iter=1000)
cv_results = cross_validate(lr, X_a, y_a, cv=5, scoring=best_cutoff_scorer)
mean_cutoff_lr_a = np.mean(cv_results['test_score'])
print('Optimal cutoff for LR is {}'.format(mean_cutoff_lr_a))

Optimal cutoff for GBC is -0.7101771961530015
Optimal cutoff for LR is -0.7875295711258137


In [100]:
# NEURODEGENERATION
# Hyperparameter search for best cutoff for F1
best_cutoff_scorer = metrics.make_scorer(best_cutoff, needs_threshold=True)
gbc = GradientBoostingClassifier(random_state=1995, max_depth=1, n_estimators=150)
cv_results = cross_validate(gbc, X_n, y_n, cv=5, scoring=best_cutoff_scorer)
mean_cutoff_gb_n = np.mean(cv_results['test_score'])
print('Optimal cutoff for GBC is {}'.format(mean_cutoff_gb_n))

best_cutoff_scorer = metrics.make_scorer(best_cutoff, needs_threshold=True)
lr = LogisticRegression(random_state=1995, max_iter=1000)
cv_results = cross_validate(lr, X_n, y_n, cv=5, scoring=best_cutoff_scorer)
mean_cutoff_lr_n = np.mean(cv_results['test_score'])
print('Optimal cutoff for LR is {}'.format(mean_cutoff_lr_n))

Optimal cutoff for GBC is -1.0265091007812368
Optimal cutoff for LR is -0.9537148966513527


### Evaluation

In [101]:
# Train final model
clf_lr_n = LogisticRegression(random_state=1995,max_iter=1000).fit(X_n,y_n)
clf_gb_n = GradientBoostingClassifier(random_state=1995, max_depth=1, n_estimators=150).fit(X_n,y_n)

clf_lr_a = LogisticRegression(random_state=1995,max_iter=1000).fit(X_a,y_a)
clf_gb_a = GradientBoostingClassifier(random_state=1995, max_depth=1, n_estimators=150).fit(X_a,y_a)

In [102]:
# Evaluate Neurodegeneration
keys = ['SUVR.Schaefer200.ROI.idx.{}'.format(i) for i in range(1,201)]
keys.extend(['Age_x','Sex OH','APOE A1_x','APOE A2_x'])
#X_n_test = out_GM_volume_df_test[['tau.global.SUVR','Age_x','Sex OH','APOE A1_x','APOE A2_x']].values
X_n_test = out_GM_volume_df_test[keys].values
y_n_test = out_GM_volume_df_test[['N+']].values.flatten()

print('Performance LR:')
y_hat_n_lr_test = evaluate_performance(clf_lr_n, X_n, y_n, X_n_test, y_n_test, mean_cutoff_lr_n)
print('\n\nPerformance GB:')
y_hat_n_gb_test = evaluate_performance(clf_gb_n, X_n, y_n, X_n_test, y_n_test, mean_cutoff_gb_n)

Performance LR:
Prediction time was 0.00037360191345214844
Train
F1: 0.6588921282798834
AUC: 0.7524398797595191
---
Test
F1: 0.5287356321839081 | Precision: 0.5 | Recall: 0.5609756097560976
AUC: 0.7911494085000731


Performance GB:
Prediction time was 0.0009856224060058594
Train
F1: 0.712166172106825
AUC: 0.7829659318637275
---
Test
F1: 0.5263157894736842 | Precision: 0.46296296296296297 | Recall: 0.6097560975609756
AUC: 0.7444136118007886


In [69]:
lr_n = pd.DataFrame({'gt':y_n_test ,'pred':clf_lr_n.decision_function(X_n_test)})
lr_n.to_csv('../data/lr_neurodegen.csv')

gb_n = pd.DataFrame({'gt':y_n_test ,'pred':clf_gb_n.decision_function(X_n_test)})
gb_n.to_csv('../data/gb_neurodegen.csv')

In [64]:
# Evaluate Amyloid
out_abeta_df_test = tau_abeta_FBB_overlap_suvr_df.iloc[test_ids_amyloid_overlap_gmdensity_indices]
#out_abeta_df_test = tau_abeta_FBB_overlap_suvr_df

keys = ['SUVR.DKT.ROI.idx.{}_tau'.format(i) for i in dktLabels]
keys.extend(['Age','Sex OH','APOE A1','APOE A2'])
X_a_test = out_abeta_df_test[keys].values
y_a_test = out_abeta_df_test[['A+']].values.flatten()

print('Performance LR:')
y_hat_a_lr_test = evaluate_performance(clf_lr_a, X_a, y_a, X_a_test, y_a_test, mean_cutoff_lr_a)
print('\n\nPerformance GB:')
y_hat_a_gb_test = evaluate_performance(clf_gb_a, X_a, y_a, X_a_test, y_a_test, mean_cutoff_gb_a)

Performance LR:
Prediction time was 0.0003578662872314453
Train
F1: 0.7062146892655367
AUC: 0.767746913580247
---
Test
F1: 0.6928104575163399 | Precision: 0.6883116883116883 | Recall: 0.6973684210526315
AUC: 0.8672157492969064


Performance GB:
Prediction time was 0.0007977485656738281
Train
F1: 0.7813411078717202
AUC: 0.8211111111111111
---
Test
F1: 0.7236842105263158 | Precision: 0.7236842105263158 | Recall: 0.7236842105263158
AUC: 0.8502410606669345


In [70]:
lr_a = pd.DataFrame({'gt':y_a_test ,'pred':clf_lr_a.decision_function(X_a_test)})
lr_a.to_csv('../data/lr_amyloid.csv')

gb_a = pd.DataFrame({'gt':y_a_test ,'pred':clf_gb_a.decision_function(X_a_test)})
gb_a.to_csv('../data/gb_amyloid.csv')

In [65]:
# Ensure test sets are looking at same subjects (last three columns are sex and APOE status)
assert np.all(X_a_test[:,-3:]==X_n_test[:,-3:]), 'Test sets are different between N and A'

### ATN Classification

In [66]:
# Logistic Regression
gt = classify_atn(y_n_test,y_a_test)
pred = classify_atn(y_hat_n_lr_test,y_hat_a_lr_test)
acc = metrics.accuracy_score(gt,pred)
f1 = metrics.f1_score(gt,pred,average='macro')
prec = metrics.precision_score(gt,pred,average='macro')
recall = metrics.recall_score(gt,pred,average='macro')
print('For ATN classification with LR, accuracy was {} and F1 was {} (Prec: {} | Rec: {})'.format(acc,f1,prec,recall))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# Gradient Boosting
gt = classify_atn(y_n_test,y_a_test)
pred = classify_atn(y_hat_n_gb_test,y_hat_a_gb_test)
acc = metrics.accuracy_score(gt,pred)
f1 = metrics.f1_score(gt,pred,average='macro')
prec = metrics.precision_score(gt,pred,average='macro')
recall = metrics.recall_score(gt,pred,average='macro')
print('For ATN classification with GB, accuracy was {} and F1 was {} (Prec: {} | Rec: {})'.format(acc,f1,prec,recall))

For ATN classification with GB, accuracy was 0.7101449275362319 and F1 was 0.5324228781831122 (Prec: 0.5623778998778999 | Rec: 0.5453830891330891)


In [ ]:
n_pred_gb = [str(int(i)) for i in y_hat_n_gb_test.tolist()]
n_pred_lr = [str(int(i)) for i in y_hat_n_lr_test.tolist()]
print(','.join(n_pred_gb))
print('---')
print(','.join(n_pred_lr))

0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0
---
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0


In [ ]:
a_pred_gb = [str(int(i)) for i in y_hat_a_gb_test.tolist()]
a_pred_lr = [str(int(i)) for i in y_hat_a_lr_test.tolist()]
print(','.join(a_pred_gb))
print('---')
print(','.join(a_pred_lr))

0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,1,0,1,1,0,1,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,1,1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,1,1,1,1,0,0,1,0,0,0,0,1,1,1,0
---
0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,1,1,0,1,1,0,1,1,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,1,1,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,1,1,0,0,1,1,0,1,1,1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0,1,0,1,1,0


In [ ]:
# Accuracy is a bad measure when you have such a small minority 
correct = 0
miss = 0
double_miss = 0
abeta_miss = 0
n_miss = 0

out_a = y_hat_a_lr_test
out_n = y_hat_n_lr_test

for i in range(len(y_a_test)):
    if (y_a_test[i]==out_a[i]) and (y_n_test[i]==out_n[i]):
        correct += 1
    elif (y_a_test[i]!=out_a[i]) and (y_n_test[i]!=out_n[i]):
        double_miss += 1
    else:
        if (y_a_test[i]!=out_a[i]):
            abeta_miss += 1
        else:
            n_miss += 1
        miss += 1

print("Out of {} test subjects, {} were ATN classified correctly, {} had one predicted biomarker incorrect and {} had both biomarkers incorrect".format(len(y_a_test),correct,miss,double_miss))
print("Abeta miss only: {}".format(abeta_miss))
print("N miss only: {}".format(n_miss))        

Out of 207 test subjects, 142 were ATN classified correctly, 52 had one predicted biomarker incorrect and 13 had both biomarkers incorrect
Abeta miss only: 34
N miss only: 18
